In [3]:
import comet_ml
from config import SAVED_MODEL

comet_ml.init(api_key="wfYwSiDlqTHVyTWHjAuT6qI0P")

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline, BertForMaskedLM, BertTokenizer
from transformers import BertModel
from datasets import load_dataset

from torch.utils.data import DataLoader

from transformers import Trainer, TrainingArguments
from datetime import datetime


COMET WARNING: comet_ml.init() is deprecated and will be removed soon. Please use comet_ml.login()
COMET INFO: Valid Comet API Key saved in /home/henryp/.comet.config (set COMET_CONFIG to change where it is saved).


Model

In [4]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
# model = BertModel.from_pretrained(model_name)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

Data


In [5]:
dataset = load_dataset("imdb")  # or your own dataset
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")  # required by HF
tokenized_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

train_loader = DataLoader(tokenized_dataset['train'], batch_size=8, shuffle=True)
val_loader = DataLoader(tokenized_dataset['test'], batch_size=8)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir='./logs',
    report_to="comet_ml",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    data_collator=data_collator,
)

timestamp = datetime.now().strftime('%y%m%d%H%M')
model_path = f"{SAVED_MODEL}/{timestamp}/{model_name}"
print(f"Will train model and save it to {model_path}")
trainer.train()

/home/henryp/venv/llms/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch, keras, tensorflow.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


Will train model and save it to ./saved_model/2505071143/bert-base-uncased


COMET INFO: Experiment is live on comet.com https://www.comet.com/phillhenry/general/f3f125fa11e34b3fbb41948b4a398fd7



In [ ]:
#model_path = f"{MyLlamaModel.base_output_dir}/{model_name}"

model.save_pretrained(model_path)
tokenizer.save_pretrained(f"{model_path}_tokenizer")
tokenizer.save_vocabulary(f"{model_path}_vocab")
#model.save_pretrained_merged(model_path, tokenizer=tokenizer) # merged_4bit_forced
print("finished")

In [ ]:
unmasker = pipeline('fill-mask', model=model)
print(unmasker("Hello I'm a [MASK] model."))